In [3]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import pandas as pd


In [5]:
scores = pd.read_csv('../data/scores.csv')

In [8]:
scores.head()

,id,user_id,movie_id,rating,Date,user_id_scaled,movie_id_scaled,rating_scaled
0,0,196,242,3,1997-12-04 15:55:49,-0.999519,-0.554812,-0.470707
1,1,186,302,3,1998-04-04 19:22:22,-1.037026,-0.373432,-0.470707
2,2,22,377,1,1997-11-07 07:18:36,-1.652150,-0.146707,-2.247430
3,3,244,51,2,1997-11-27 05:02:03,-0.819482,-1.132206,-1.359069
4,4,166,346,1,1998-02-02 05:33:16,-1.112041,-0.240420,-2.247430


In [9]:
scaler = StandardScaler()
scores[['user_id_scaled', 'movie_id_scaled', 'rating_scaled']] = scaler.fit_transform(scores[['user_id', 'movie_id', 'rating']])


In [10]:
scores.head()

,id,user_id,movie_id,rating,Date,user_id_scaled,movie_id_scaled,rating_scaled
0,0,196,242,3,1997-12-04 15:55:49,-0.999519,-0.554812,-0.470707
1,1,186,302,3,1998-04-04 19:22:22,-1.037026,-0.373432,-0.470707
2,2,22,377,1,1997-11-07 07:18:36,-1.652150,-0.146707,-2.247430
3,3,244,51,2,1997-11-27 05:02:03,-0.819482,-1.132206,-1.359069
4,4,166,346,1,1998-02-02 05:33:16,-1.112041,-0.240420,-2.247430
